In [0]:
from pyspark.sql.functions import col, to_date
from pyspark.sql.types import DateType

# Tablas
tabla_bronze = "main.weather.bronze_distritos_diarios"
tabla_silver = "main.weather.silver_distritos_diarios"

# Leer Bronze
df_bronze = spark.table(tabla_bronze).withColumn("fecha", to_date(col("fecha"), "yyyy-MM-dd"))

# Eliminar duplicados por clave natural
df_bronze = df_bronze.dropDuplicates(["distrito", "fecha"])

# Verificar existencia de tabla Silver
if spark.catalog.tableExists(tabla_silver):
    df_silver = spark.table(tabla_silver)

    # Detectar nuevos registros por combinación clave (distrito, fecha)
    df_nuevos = df_bronze.join(df_silver, ["distrito", "fecha"], "left_anti")
else:
    df_nuevos = df_bronze

# Insertar nuevos datos si los hay
if df_nuevos.count() > 0:
    df_nuevos.write.format("delta").mode("append").saveAsTable(tabla_silver)
    print(f"✅ {df_nuevos.count()} nuevos registros insertados en Silver.")
else:
    print("ℹ️ No se encontraron nuevos registros para insertar en Silver.")
